Checking the committee Atomic energies qoi

In [ ]:
using Revise
using Pkg; Pkg.activate(".")

In [ ]:
using Unitful
using PotentialLearning
using Random: randperm
using JLD2
using InteratomicPotentials
using AtomsBase, AtomsCalculators
using Statistics
using CairoMakie, ColorSchemes
using LinearAlgebra

In [ ]:
includet("../files/conformal_prediction_utils.jl")
includet("../files/committee_potentials.jl")
includet("../files/committee_qois.jl")

In [ ]:
training_data_dict = load("training_data.jld2")
frenkel_train_ds = training_data_dict["frenkel_train_ds"]
pristine_train_ds = training_data_dict["pristine_train_ds"]

In [ ]:
calibtest_datasets = load("../cesmix_prez/datasets_with_descriptors.jld2")
pristine_base_calib_ds = calibtest_datasets["pristine_base_calib_ds"]
pristine_base_test_ds = calibtest_datasets["pristine_base_test_ds"]
frenkel_base_calib_ds = calibtest_datasets["frenkel_base_calib_ds"]
frenkel_base_test_ds = calibtest_datasets["frenkel_base_test_ds"]

In [ ]:
combined_calib_ds = concat_dataset([pristine_base_calib_ds; frenkel_base_calib_ds])
combined_test_ds = concat_dataset([pristine_base_test_ds; frenkel_base_test_ds])

In [ ]:
ace_cmte_dict = load("../cesmix_prez/ace_cmte1.jld2")
ensemble_members = ace_cmte_dict["members"]

In [ ]:
ace = ACE(species            = [:Hf],
          body_order         = 4,
          polynomial_degree  = 10,
          wL                 = 1.5,
          csp                = 1.0,
          r0                 = 2.15,
          rcutoff            = 5.0)

In [ ]:
my_cmte = CommitteePotential(ensemble_members; energy_units=u"eV", length_units=u"Å")
cmte_energy = CmteEnergy(Statistics.std, strip_units=true)

In [ ]:
get_values(get_local_descriptors(pristine_train_ds[1]))

In [ ]:
check_configs = load_data("./data/pristine_1.xyz", ExtXYZ(u"eV", u"Å"))

In [ ]:
get_local_descriptors(check_configs[1])

In [ ]:
all_atomic_energies = compute_all_atomic_energies(pristine_train_ds[1], my_cmte)

In [ ]:
sum.(all_atomic_energies)

In [ ]:
compute_all_energies(pristine_train_ds[1],my_cmte) # note this is also just a different configuration

In [ ]:
trial_atomic_energies = compute_all_atomic_energies(check_configs[1], my_cmte)

In [ ]:
mapslices(Statistics.var, stack(trial_atomic_energies), dims=2)

In [ ]:
maximum(Statistics.var([ae[1] for ae in trial_atomic_energies]))

In [ ]:
atomic_energies_qoi = CmteAtomicEnergies(Statistics.var, true)

In [ ]:
compute(atomic_energies_qoi, check_configs[1], my_cmte)